In [8]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install -r requirements.txt 

  Obtaining dependency information for anyio==4.0.0 from https://files.pythonhosted.org/packages/36/55/ad4de788d84a630656ece71059665e01ca793c04294c463fd84132f40fe6/anyio-4.0.0-py3-none-any.whl.metadata
  Using cached anyio-4.0.0-py3-none-any.whl.metadata (4.5 kB)
  Obtaining dependency information for filelock==3.12.3 from https://files.pythonhosted.org/packages/52/90/45223db4e1df30ff14e8aebf9a1bf0222da2e7b49e53692c968f36817812/filelock-3.12.3-py3-none-any.whl.metadata
  Using cached filelock-3.12.3-py3-none-any.whl.metadata (2.7 kB)
  Obtaining dependency information for fsspec==2023.9.0 from https://files.pythonhosted.org/packages/3a/9f/b40e8e5be886143379000af5fc0c675352d59e82fd869d24bf784161dc77/fsspec-2023.9.0-py3-none-any.whl.metadata
  Using cached fsspec-2023.9.0-py3-none-any.whl.metadata (6.7 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Obtaining dependency information for httpcore==0.18.0 from https://files.pythonhosted.org/packages/ac/97/724afbb7925339f6214bf1fdb5

In [3]:
pip install "psycopg[binary,pool]"


[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: /home/jovyan/resources/venv/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [26]:
import json
import os
from datetime import datetime
from uuid import uuid4

import openai
import pandas as pd
# from dotenv import dotenv_values

import pprint
import random
from copy import deepcopy
from dataclasses import asdict, dataclass
from datetime import datetime, timezone
from itertools import combinations
from typing import Any, Dict, List, Union

import numpy as np
import pandas as pd
import plotly.express as px
import psycopg
from pandas import option_context
from psycopg.rows import dict_row
from collections import defaultdict
from openai_info import OPENAI_API_KEY, OPENAI_ORGANIZATION #
import httpx


In [27]:
pd.options.display.float_format = "{:.2f}".format
pd.options.display.max_rows = None

pp = pprint.PrettyPrinter(indent=4)

In [28]:
def read_json(json_path):
    json_data = None
    try:
        with open(json_path, "r") as json_fp:
            json_data = json.load(json_fp)
    except Exception as e:
        print(e)
    return json_data


def write_json(data: Union[Dict, List], file_name: str) -> str:
    out_file_path = os.path.abspath(file_name)
    with open(out_file_path, "w") as json_fp:
        json.dump(data, json_fp)
    return out_file_path

In [29]:
random_tasks = read_json("random_tasks.json")

In [30]:
openai.organization = OPENAI_ORGANIZATION
openai.api_key = OPENAI_API_KEY

In [31]:
# system_prompt = """
# You are an AI quality evaluator. Your role involves judging the accuracy of an AI model's response using the provided <source> text. Here are the steps:
# 1. Read the <source> text to understand the task's context.
# 2. Grasp the given <question>.
# 3. Understand the <actual_answer>, the ideal answer to the <question> using only the information from the <source>.
# 4. Examine the AI's <answer> to the same <question>.
# 5. Critically evaluate the <answer>, considering aspects like accuracy, conciseness, organization, and usefulness in relation to the <source>.
# 6. Compare the <answer> to the <actual_answer>, noting similarities and differences.
# 7. Assign a score to the <answer> on a scale of 1 to 10, where 1 means no match, and 10 means almost identical to <ground_truth>.
# 8. Provide your judgment and reasoning in a JSON format. Include "correct" (true if the <answer> aligns closely with the <ground_truth>), "score", and "reason" (explaining your score with clear evidence from the <source> and <answer>). An example: {"correct": true, "score": 8, "reason": "The answer is concise, accurate, and follows the source closely."}.
# """

In [32]:
#alternate system prompt that assigns a letter grade instead of a value (0-10). There are also 10 possible values here, but maybe the judge might be more intentional about picking a letter vs a number.
system_prompt = """
As an AI quality evaluator, your role is to gauge the accuracy of an AI model's response using the provided <source> text. Follow these steps:

1.) Understand the context by reading the <source> text.
2.) Comprehend the given <question>.
3.) Determine the <actual_answer>—the ideal answer using only the <source> information.
4.) Review the AI's <answer>.
5.) Analyze the <answer>'s accuracy, brevity, structure, and utility in relation to the <source>.
6.) Contrast the <answer> and the <actual_answer>.
7.) Assign a grade to the <answer> using A+ to F- scale, A+ being nearly identical to <ground_truth> and F- being entirely dissimilar.
8.) Document your judgment and reasoning in JSON format: include "correct" (true if <answer> aligns closely with <ground_truth>), "score", and "reason" (justifying your score with clear evidence from <source> and <answer>).
Note: Favor responses that closely align with the source and discourage those incorporating outside information. The aim is to evaluate the AI's understanding of the provided source, not its knowledge from training data.
"""

In [33]:
async def ask_llm(
    prompt_text: str, llm_endpoint: str, client: httpx.AsyncClient
):
    headers = {"Content-Type": "application/json"}
    data = {
        "inputs": f"<|prompt|>{prompt_text}<|endoftext|><|answer|>",
        "parameters": {
            "max_new_tokens": 1024,
            "min_new_tokens": 2,
            "num_beams": 1,
            "do_sample": False,
            "temperature": 0.3,
            "repetition_penalty": 1.2,
            "renormalize_logits": True,
        },
    }

    try:
        response = await client.post(
            llm_endpoint,
            json=data,
            headers=headers,
        )
        response.raise_for_status()
    except httpx.RequestError as exc:
        print(f"An error occurred while requesting {exc.request.url!r}.")
    except httpx.HTTPStatusError as exc:
        print(
            f"Error response {exc.response.status_code} while requesting {exc.request.url!r}."
        )
    else:
        try:
            response_text = response.json()["generated_text"]
        except Exception as e:
            print(e)
        else:
            return response_text

In [34]:
async def answer_questions(json_file: str, llm_endpoint: str, client: httpx.AsyncClient):
    with open(json_file, 'r') as file:
        data = json.load(file)

    answer_data = []
    for entry in data:
        prompt_text = f""" {entry['source']} Given the information above and only the information from above, answer the following question {entry['question']} """
        answer_text = await ask_llm(prompt_text, llm_endpoint, client)

        if answer_text:
            answer_data.append({'id': entry['id'], 'answer': answer_text})

    with open('answers.json', 'w') as file:
        json.dump(answer_data, file)

In [39]:
async def judge_questions(system_prompt, eval_model_name, prompt_json, answer_json):    
    with open(prompt_json, 'r') as file:
        prompt_data = json.load(file)
    
    with open(answer_json, 'r') as file:
        answer_data = json.load(file)
        
    eval_data = []
    results = []
        
    for i in range(len(prompt_data)):
        prompt_entry = prompt_data[i]
        answer_entry = answer_data[i]
        template = f"""
        <source>
        {prompt_entry['source']}
        </source>

        <question>
        {prompt_entry['question']}
        </question>

        <actual_answer>
        {prompt_entry['answer']}
        </actual_answer>

        <answer>
        {answer_entry['answer']}
        </answer>
        """

        chat_completion_resp = await openai.ChatCompletion.acreate(
            model=eval_model_name,
            messages=[
                {
                    "role": "system",
                    "content": system_prompt,
                },
                {
                    "role": "user",
                    "content": template,
                },
            ],
            temperature=0.1,
            max_tokens=1024,
        )
        if chat_completion_resp.choices[0].message.content:
            eval_data.append({'id': answer_entry['id'], 'eval_result': chat_completion_resp.choices[0].message.content})
            results.append({'id': prompt_entry['id'], 'source': prompt_entry['source'], 'question': prompt_entry['question'], 'ground truth': prompt_entry['answer'], 'answer': answer_entry['answer']})

    with open('eval_marco.json', 'w') as file:
        json.dump(eval_data, file)
    with open('results_marco.json', 'w') as file:
        json.dump(results, file)

In [40]:
model_name = "h2oai/h2ogpt-gm-oasst1-en-2048-falcon-40b-v1"
eval_model_name = "gpt-4-0613"
llm_endpoint = "https://chat-large.api.h2o.ai/generate"
auth = httpx.BasicAuth("user", "bhx5xmu6UVX4")
client = httpx.AsyncClient(auth=auth, timeout=None)

In [41]:
await answer_questions("prompts_marco.json", llm_endpoint, client)

In [42]:
eval_model_name = "gpt-4-0613"
await judge_questions(system_prompt, eval_model_name, "prompts_marco.json", "answers.json")